# Insurance Challenge 1 - Baseline Submission

This notebook provides a simple baseline for **Insurance Challenge 1: Claims Complexity Prediction**.

**Goal**: Predict `ClaimComplexityLabel` (Simple/Moderate/Complex) for insurance claims
**Metric**: Macro-F1 Score - Higher is better

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses only tabular data with a simple Random Forest classifier.

In [ ]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from agentds import BenchmarkClient

# 🔑 REPLACE WITH YOUR CREDENTIALS
client = BenchmarkClient(
    api_key="your-api-key-here",        # Get from your team dashboard
    team_name="your-team-name-here"     # Your exact team name
)

# Load data from PVC paths
print("📂 Loading Insurance Challenge 1 data...")

# Load training data
train_claims = pd.read_csv("/home/jovyan/shared/datasets/Insurance/train_claims.csv")
train_policies = pd.read_csv("/home/jovyan/shared/datasets/Insurance/train_policies_subset.csv")

# Load test data  
test_claims = pd.read_csv("/home/jovyan/shared/datasets/Insurance/test_claims.csv")
test_policies = pd.read_csv("/home/jovyan/shared/datasets/Insurance/test_policies_subset.csv")

print(f"✅ Data loaded:")
print(f"   Train claims: {train_claims.shape}")
print(f"   Train policies: {train_policies.shape}")
print(f"   Test claims: {test_claims.shape}")
print(f"   Test policies: {test_policies.shape}")

In [ ]:
# 2. Tabular-Only Baseline Model and Predictions

# Merge claims with policies for training
train_data = train_claims.merge(train_policies, on='PolicyID', how='left')
test_data = test_claims.merge(test_policies, on='PolicyID', how='left')

# Select specific numeric features from the actual data
# From inspection: ReportedDamage, NumParties, HolderAge, AnnualMileage, LocationUrban, CreditScore
claim_features = ['ReportedDamage', 'NumParties']
policy_features = ['HolderAge', 'AnnualMileage', 'LocationUrban', 'CreditScore']
all_features = claim_features + policy_features

print(f"📊 Using {len(all_features)} features for baseline:")
print(f"   Claim features: {claim_features}")
print(f"   Policy features: {policy_features}")

# Prepare training data
X_train = train_data[all_features].fillna(0)  # Simple imputation
y_train = train_data['ClaimComplexityLabel']

# Prepare test data
X_test = test_data[all_features].fillna(0)

# Train simple Random Forest classifier baseline
print("🤖 Training Random Forest classifier...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)



# Create submission file
submission_df = pd.DataFrame({
    'ClaimID': test_claims['ClaimID'],
    'ClaimComplexityLabel': predictions
})

# Save predictions
submission_df.to_csv("insurance_challenge1_predictions.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")


In [ ]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Insurance", 1, "insurance_challenge1_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. Move on to Insurance Challenge 2!")